# Querying

This notebook demonstrates how to retrieve, query and search data using the Forge.

In [2]:
from kgforge.core import KnowledgeGraphForge

In [3]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

## Imports

In [4]:
from kgforge.core import Resource

## Retrieval

In [5]:
jane = Resource(type="Person", name="Jane Doe")

In [6]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [7]:
resource = forge.retrieve(jane.id)

In [8]:
resource == jane

True

### specific version

In [9]:
jane = Resource(type="Person", name="Jane Doe")

In [10]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [11]:
forge.tag(jane, "v1")

<action> _tag_one
<succeeded> True


In [12]:
jane.email = "jane.doe@epfl.ch"

In [13]:
forge.update(jane)

<action> _update_one
<succeeded> True


In [14]:
jane._store_metadata.version

2

In [15]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [16]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [17]:
jane_v1 == jane_v1_tag

True

In [18]:
print(jane_v1)

{
    id: 9e689d89-3425-46d8-95e7-7058e9724ea4
    type: Person
    name: Jane Doe
}


### error handling

In [19]:
resource = forge.retrieve("123")

<action> retrieve
<error> RetrievalError: resource not found



In [20]:
print(resource)

None


## Searching

In [21]:
jane = Resource(type="Person", name="Jane Doe")

In [22]:
john = Resource(type="Person", name="John Smith")

In [23]:
association_jane = Resource(type="Association", agent=jane)

In [24]:
association_john = Resource(type="Association", agent=john)

In [25]:
associations = [association_jane, association_john]

In [26]:
forge.register(associations)

<count> 2
<action> _register_one
<succeeded> True


`Association` is a known type by the Model, and the `paths` method will load the data structure for the given type. Refer to the `11 - Modeling.ipynb` notebook to learn about Modeling and Types.

In [27]:
p = forge.paths("Association")

<info> DemoModel does not distinguish values and constraints in templates for now.
<info> DemoModel does not automatically include nested schemas for now.


You have autocompletion on `p` and this can be used to build a search.

In [28]:
resources = forge.search(p.type == "Association")

<info> DemoStore does not support handling of errors with QueryingError for now.
<info> DemoStore does not support traversing lists for now.


In [29]:
type(resources)

list

In [30]:
len(resources)

2

In [31]:
type(resources[0])

kgforge.core.resource.Resource

In [32]:
forge.as_dataframe(resources)

,id,type,agent.type,agent.name
0,40cf0989-175a-4948-ae49-014546efbd8e,Association,Person,Jane Doe
1,b5c39b73-cc11-4339-8c70-47f802f8ddaf,Association,Person,John Smith


In [33]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,agent.type,agent.name,deprecated,version
0,40cf0989-175a-4948-ae49-014546efbd8e,Association,Person,Jane Doe,False,1
1,b5c39b73-cc11-4339-8c70-47f802f8ddaf,Association,Person,John Smith,False,1


### nested field querying

You have autocompletion on `p` but also on nested properties like `p.agent`.

In [34]:
resources = forge.search(p.type == "Association", p.agent.name == "John Smith")

<info> DemoStore does not support handling of errors with QueryingError for now.
<info> DemoStore does not support traversing lists for now.


In [35]:
len(resources)

1

In [36]:
# print(resources[0])

## Graph traversing

SPARQL is used to traverse the graph. The user can provide simplified queries: no prefixes, no compacted URIs, nor full URIs is required, only the term (property) available in the Model to the user.  Refer to the `11 - Modeling.ipynb` notebook to learn about Templates.

In [37]:
jane = Resource(type="Person", name="Jane Doe")

In [38]:
john = Resource(type="Person", name="John Smith")

In [39]:
association = Resource(type="Association", agent=[jane, john])

In [40]:
forge.register(association)

<action> _register_one
<succeeded> True


In [41]:
forge.template("Association")

<info> DemoModel does not distinguish values and constraints in templates for now.
<info> DemoModel does not automatically include nested schemas for now.
{
    type: Association
    agent:
    {
        type: Person
        name: hasattr
    }
}


In [42]:
query = """
    SELECT ?x ?name
    WHERE {
        ?x a Association ;
           agent ?agent .
        ?agent name ?name .
    }
"""

In [43]:
# resources = forge.sparql(query)

In [44]:
# type(resources)

In [45]:
# len(resources)

In [46]:
# type(resources[0])

### rewritten query display

In [47]:
forge.sparql(query, debug=True)

Submitted query:
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX schema: <http://schema.org/>
   
       SELECT ?x ?name
       WHERE {
           ?x rdf:type prov:Association ;
              prov:agent ?agent .
           ?agent schema:name ?name .
       }

<action> not_supported
<error> NotSupportedError: DemoStore is not supporting _sparql()



## Downloading

In [48]:
jane = Resource(type="Person", name="Jane Doe")

In [49]:
! ls -p ../../data | egrep -v /$

associations.tsv
persons.csv


In [50]:
distribution = forge.attach("../../data")

In [51]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [52]:
# forge.register(association)

In [53]:
# forge.download(association, "distribution.contentUrl", "./downloaded/")

In [54]:
# ! ls ./downloaded/